In [89]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [90]:
data = pd.read_csv('https://lms.skillfactory.ru/assets/courseware/v1/1d68c628a08cd5521ae70e976577568b/asset-v1:Skillfactory+DST-PRO+15APR2020+type@asset+block/movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
586,tt0212985,87000000,351692268,Hannibal,Anthony Hopkins|Julianne Moore|Gary Oldman|Ray...,Ridley Scott,"His genius undeniable. His evil, unspeakable.",After having successfully eluded the authoriti...,131,Crime|Drama|Thriller|Horror,Universal Pictures|Scott Free Productions|Metr...,2/8/2001,6.3,2001
1515,tt0414853,51000000,72779000,Barnyard,Courteney Cox|Kevin James|Sam Elliott|Danny Gl...,Steve Oedekerk,What happens in the barn stays in the barn,"When things get crazy at the farm, it's up to ...",90,Animation|Comedy|Family,Paramount Pictures|Nickelodeon Movies|Paramoun...,8/4/2006,5.3,2006
1867,tt0181536,43000000,80049764,Finding Forrester,Sean Connery|Rob Brown|F. Murray Abraham|Anna ...,Gus Van Sant,"In an ordinary place, he found the one person ...",Gus van Sant tells the story of a young Africa...,136,Drama,Fountainbridge Films|Laurence Mark Productions...,12/21/2000,6.9,2000
998,tt1024648,44500000,232324128,Argo,Ben Affleck|Bryan Cranston|Alan Arkin|John Goo...,Ben Affleck,The movie was fake. The mission was real.,As the Iranian revolution reaches a boiling po...,120,Drama|Thriller,GK Films|Warner Bros.|Smokehouse Pictures,10/11/2012,7.0,2012
706,tt0482603,37000000,63800345,Space Chimps,Andy Samberg|Cheryl Hines|Jeff Daniels|Stanley...,Kirk De Micco,"Houston, we have a problem!",Circus monkey Ham III works in a circus where ...,81,Animation|Family,Starz Media|Starz Animation|Odyssey Entertainm...,7/17/2008,5.1,2008


In [91]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [92]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [93]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

In [94]:
data['profit'] = data.revenue - data.budget

In [95]:
def count_genres(genres_str, counts_dict):
    for genre in genres_str.split('|'):
        if genre in counts_dict.keys():
            counts_dict[genre] += 1
        else:
            counts_dict[genre] = 1

In [96]:
# making a new df splitting rows by directors
data["director"] = data.director.str.split('\|')
data_directors = data.explode('director')
data_directors.reset_index(inplace=True)

In [97]:
# making a new df splitting rows by actors
data["actor"] = data.cast.str.split('\|').apply(lambda x: sorted(x) if isinstance(x, list) else x)
data_cast = data.explode('actor')
data_cast.reset_index(inplace=True)

In [98]:
# making a new df splitting rows by genres
data["genre"] = data.genres.str.split('\|')
data_genres = data.explode('genre')
data_genres.reset_index(inplace=True)

In [99]:
# making a new df splitting rows by production companies
data["company"] = data.production_companies.str.split('\|')
data_companies = data.explode('company')
data_companies.reset_index(inplace=True)

In [100]:
data["month"] = data.release_date.str.extract(r'^(\d+)\/\d+\/\d*$', expand=False)

In [101]:
seasons = {
    1: 'winter',
    2: 'winter',
    3: 'spring',
    4: 'spring',
    5: 'spring',
    6: 'summer',
    7: 'summer',
    8: 'summer',
    9: 'fall',
    10: 'fall',
    11: 'fall',
    12: 'winter',
}
data['season'] = data.month.apply(lambda x: seasons[int(x)])

In [102]:
data_directors["month"] = data_directors.release_date.str.extract(r'^(\d+)\/\d+\/\d*$', expand=False)
data_directors['season'] = data_directors.month.apply(lambda x: seasons[int(x)])

In [103]:
data_companies['title_length'] = data_companies.original_title.str.len()

In [104]:
data_companies['overview_words'] = data_companies.overview.str.count(r'\w+')

In [105]:
# pairs of actors in movies to find the most frequent one
data['actors_combinations'] = data.actor.apply(lambda x: list(combinations(x, 2)))

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [106]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [107]:
# тут пишем ваш код для решения данного вопроса:
data[data.budget == data.budget.max()][['original_title', 'imdb_id']]

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


ВАРИАНТ 2

In [108]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [109]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)'

In [110]:
data[data.runtime == data.runtime.max()][['original_title', 'imdb_id']]

,original_title,imdb_id
1157,Gods and Generals,tt0279111


# 3. Какой из фильмов самый короткий (в минутах)?





In [111]:
data[data.runtime == data.runtime.min()][['original_title', 'imdb_id']]

,original_title,imdb_id
768,Winnie the Pooh,tt1449283


In [112]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [113]:
data.runtime.mean()

109.6585494970884

In [114]:
answers['4'] = '2. 110'

# 5. Каково медианное значение длительности фильмов? 

In [115]:
data.runtime.median()

107.0

In [116]:
answers['5'] = '1. 107'

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [117]:
# лучше код получения столбца profit вынести в Предобработку что в начале
data[data.profit == data.profit.max()][['original_title', 'imdb_id']]

,original_title,imdb_id
239,Avatar,tt0499549


In [118]:
answers['6'] = '5. Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [119]:
data[data.profit == data.profit.min()][['original_title', 'imdb_id']]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


In [120]:
answers['7'] = '5. The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [121]:
len(data[data.profit > 0])

1478

In [122]:
answers['8'] = '1. 1478'

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [123]:
a = data[(data.release_year == 2008)]
a[a.revenue == a.revenue.max()][['original_title', 'imdb_id']]

,original_title,imdb_id
599,The Dark Knight,tt0468569


In [124]:
answers['9'] = '4. The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [125]:
b = data[(data.release_year <= 2014) & (data.release_year >= 2012)]
b[b.profit == b.profit.min()][['original_title', 'imdb_id']]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


In [126]:
answers['10'] = '5. The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [127]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
genres = []

for genres_str in data.genres:
    for genre in genres_str.split('|'):
        if genre not in genres:
            genres.append(genre)

counts = dict()
for genre in genres:
    counts[genre] = len(data[data.genres.str.contains(genre)])

max(counts, key=counts.get)

'Drama'

ВАРИАНТ 2

In [128]:
counts = dict()
data.genres.apply(count_genres, counts_dict=counts)
max(counts, key=counts.get)

'Drama'

In [129]:
answers['11'] = '3. Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [130]:
counts = dict()
data[data.profit > 0].genres.apply(count_genres, counts_dict=counts)
max(counts, key=counts.get)

'Drama'

In [131]:
answers['12'] = '1. Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [132]:
data_directors.groupby(['director']).revenue.sum().sort_values(ascending=False).head()

director
Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Michael Bay          3886938960
J.J. Abrams          3579169916
Name: revenue, dtype: int64

In [133]:
answers['13'] = '5. Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [134]:
data_directors[data_directors.genres.str.contains('Action')].director.value_counts().head()

Robert Rodriguez      9
Paul W.S. Anderson    7
Michael Bay           7
Antoine Fuqua         6
Ridley Scott          6
Name: director, dtype: int64

In [135]:
answers['14'] = '3. Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [136]:
data_cast[data_cast.release_year == 2012].groupby(['actor']).revenue.sum().sort_values(ascending=False)[:1]

actor
Chris Hemsworth    2027450773
Name: revenue, dtype: int64

In [137]:
answers['15'] = '3. Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [138]:
data_cast[data_cast.budget > data_cast.budget.mean()].actor.value_counts()[:1]

Matt Damon    18
Name: actor, dtype: int64

In [139]:
answers['16'] = '3. Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [140]:
data_genres[data_genres.cast.str.contains('Nicolas Cage')].genre.value_counts()[:1]

Action    17
Name: genre, dtype: int64

In [141]:
answers['17'] = '2. Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [142]:
data[data.production_companies.str.contains('Paramount Pictures')].groupby(['original_title']).profit.min().sort_values(ascending=True)[:1]

original_title
K-19: The Widowmaker   -64831034
Name: profit, dtype: int64

In [143]:
answers['18'] = '1. K-19: The Widowmaker'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [144]:
data.groupby(['release_year']).revenue.sum().sort_values(ascending=False)[:1]

release_year
2015    25449202382
Name: revenue, dtype: int64

In [145]:
answers['19'] = '5. 2015'

# 20. Какой самый прибыльный год для студии Warner Bros?

In [146]:
data[data.production_companies.str.contains('Warner Bros')].groupby(['release_year']).profit.sum().sort_values(ascending=False)[:1]

release_year
2014    2295464519
Name: profit, dtype: int64

In [147]:
# It looks weird to treat some of this companies like one
data_companies[data_companies.company.str.contains('Warner')].company.value_counts()

Warner Bros.                              168
Warner Bros. Pictures                      13
Warner Bros. Animation                      5
Warner Independent Pictures (WIP)           4
Warner Bros. Interactive Entertainment      1
Name: company, dtype: int64

In [148]:
answers['20'] = '1. 2014'

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [149]:
data.month.value_counts()[:1]

9    227
Name: month, dtype: int64

In [150]:
answers['21'] = '4. Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [151]:
data.season.value_counts()['summer']

450

In [152]:
answers['22'] = '2. 450'

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [153]:
data_directors[data_directors.season == 'winter'].director.value_counts()[:1]

Peter Jackson    7
Name: director, dtype: int64

In [154]:
answers['23'] = '5. Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [155]:
data_companies.groupby('company').title_length.mean().sort_values(ascending=False)[:1]

company
Four By Two Productions    83.0
Name: title_length, dtype: float64

In [156]:
answers['24'] = '5. Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [157]:
data_companies.groupby('company').overview_words.mean().sort_values(ascending=False)[:1]

company
Midnight Picture Show    176.0
Name: overview_words, dtype: float64

In [158]:
answers['25'] = '3. Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [159]:
data[data.vote_average > data.vote_average.quantile(0.99)][['original_title', 'vote_average']]

,original_title,vote_average
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
119,Guardians of the Galaxy,7.9
125,The Imitation Game,8.0
128,Gone Girl,7.9
138,The Grand Budapest Hotel,7.9
370,Inception,7.9
599,The Dark Knight,8.1
872,The Pianist,7.9


In [160]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [161]:
data.explode('actors_combinations').actors_combinations.value_counts()

(Daniel Radcliffe, Emma Watson)        8
(Emma Watson, Rupert Grint)            8
(Daniel Radcliffe, Rupert Grint)       8
(Helena Bonham Carter, Johnny Depp)    6
(Ben Stiller, Owen Wilson)             6
                                      ..
(Ralph Fiennes, Tyler Posey)           1
(Frank Grillo, Michael Nyqvist)        1
(Andy Serkis, Ian McKellen)            1
(Jason Patric, Patrick Wilson)         1
(Frances O'Connor, Sam Robards)        1
Name: actors_combinations, Length: 17943, dtype: int64

In [162]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint '

ВАРИАНТ 2

# Submission

In [163]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint '}

In [164]:
# и убедиться что ни чего не пропустил)
len(answers)

27